In [ ]:
#!/usr/bin/env python
# coding: utf-8

# Convert HEALPix Zarr data to netCDF with variable subsets for TempestExtremes<br>
<br>
## This code compute the uivt and vivt based on ua, va, and hus<br>
<br>
## Author:<br>
- Zhe Feng || zhe.feng@pnnl.gov
- Ziming Chen || ziming.chen@pnnl.gov

In[1]:

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import xarray as xr
import os

In [ ]:
import easygems.healpix as egh
import intake

In [ ]:
plt.rcParams['figure.dpi'] = 72

### Load the catalog

In[2]:

In [ ]:
list(intake.open_catalog("https://digital-earths-global-hackathon.github.io/catalog/catalog.yaml"))

In[3]:

Load the NERSC catalog

In [ ]:
current_location = "NERSC" # "online" # 
cat = intake.open_catalog("https://digital-earths-global-hackathon.github.io/catalog/catalog.yaml")[current_location]
list(cat)

### Pick a Dataset

In[4]:

In [ ]:
s_Model = "nicam_gl11" # "icon_d3hp003" # "um_glm_n2560_RAL3p3" # "casesm2_10km_nocumulus" # "icon_ngc4008" # 
s_TimeRes = "PT6H"
#
zoom = 8
pd.DataFrame(cat[s_Model].describe()["user_parameters"])

### Load Data into a Data Set<br>
most datasets have a `zoom` parameter. We will use `zoom` level 8 [(~24km)](https://easy.gems.dkrz.de/Processing/healpix/index.html#healpix-spatial-resolution)

In[5]:

In [ ]:
ds0  = cat[s_Model](zoom=zoom, time=s_TimeRes).to_dask()
ds0  = ds0.pipe(egh.attach_coords)
ds0

In[ ]:

Variables to output<br>
# RawVarName: output_name

In [ ]:
varout_dict       = { 'time' : 'time',
                      'lat' : 'lat',
                      'lon' : 'lon',
                      'pressure': 'lev',
                      'ua': 'ua',
                      'va': 'va',
                      'hus': 'hus',
                      'orog': 'ELEV',
                      'pr' : 'pr',
                      'prs': 'prs',
                      'ps' : 'ps',
                      'psl' : 'psl',
                      'uas' : 'uas',
                      'vas' : 'vas',
                      'sfcWind' : 'sfcWind',
                      'zg' : 'zg',
                      'ta' : 'ta',
                      'tas': 'tas',
                      'rlut': 'rlut',
                     }
print(ds0.data_vars)

In[22]:

Subset variables and rename

In [ ]:
varout = []
varout_RawVarName = []
ds     = {}
for var in varout_dict:
    if var in ds0.data_vars: #  or var in ds0.coords
        ds[varout_dict[var]] = ds0[var]
        #
    else:
        print(f"No {var}: {varout_dict[var]}")

In [ ]:
ds    = xr.Dataset(ds)
print(ds)
ds.attrs = ds0.attrs
for var in ds0.coords:
    if var in varout_dict and var != varout_dict[var]:
        ds    = ds.rename_dims({var: varout_dict[var]})
        ds    = ds.rename_vars({var: varout_dict[var]})

In [ ]:
print(ds.data_vars)

 === Ziming Chen 05/11/12 ===

In [ ]:
ds['time'] = xr.decode_cf(ds).indexes['time'] #.to_datetimeindex()
ds

%%

In [ ]:
ds.data_vars

In[ ]:

Compute the uivt and vivt

Python function for vertical mass integration using xarray and numpy

In [ ]:
import numpy as np
import xarray as xr

In [ ]:
def vertical_mass_integration(hus: xr.DataArray, ps: xr.DataArray, plev: xr.DataArray) -> xr.DataArray:
    """
    Perform vertical integration of specific humidity (hus) in pressure coordinates.
    Parameters:
    - hus: xr.DataArray with dimensions (time, lev, cell)
    - ps: xr.DataArray with dimensions (time, cell), surface pressure in hPa
    - plev: xr.DataArray with dimension (lev), pressure levels in hPa
    Returns:
    - xr.DataArray with dimensions (time, cell) representing vertically integrated hus
    """
    # Ensure pressure levels are sorted from top (min) to bottom (max)
    if not np.all(np.diff(plev.values) > 0):
        hus = hus.sel(lev=plev[::-1])
        plev = plev[::-1]
    #
    if ps.max() < 1200: # hPa to Pa for ps
        ps                    = ps * 100
        ps.name               = "Pa"
        ps.attrs["units"]     = "Pa"
        ps.attrs["long_name"] = "Estimated surface pressure"
    # Mask hus where pressure level > surface pressure
    plev_3d = plev * xr.ones_like(hus)
    ps_3d = ps * xr.ones_like(hus)
    hus_masked = hus.where(plev_3d <= ps_3d)

    # Integrate using trapezoidal rule in pressure coordinates (in Pa)
    dp = np.gradient(plev.values) * 100.0  # convert hPa to Pa
    dp = xr.DataArray(dp, coords={"lev": plev}, dims=["lev"])
    dp_3d = dp * xr.ones_like(hus)
    g = 9.8  # gravity
    integrand = hus_masked * dp_3d / g
    result = integrand.sum(dim="lev")
    return result

In[ ]:

In [ ]:
out_dir = f'/pscratch/sd/w/wcmca1/scream-cess-healpix/data4TE/{s_Model}_{s_TimeRes}/'

Optional: create output directory

In [ ]:
os.makedirs(out_dir, exist_ok=True)

Clean up attributes before writing to NetCDF

In [ ]:
def clean_attrs_for_netcdf(ds):
    # Make a copy to avoid modifying the original
    attrs = ds.attrs.copy()
    
    # Handle dictionary and boolean attributes
    for key, value in list(attrs.items()):
        if isinstance(value, dict):
            # Remove dictionary attributes
            ds.attrs.pop(key, None)
        elif isinstance(value, bool):
            # Convert boolean to integer (1 for True, 0 for False)
            ds.attrs[key] = int(value)
    
    # Check all variables too
    if hasattr(ds, 'variables'):
        for var in ds.variables:
            var_attrs = ds[var].attrs.copy()
            for key, value in list(var_attrs.items()):
                if isinstance(value, dict):
                    import json
                    ds[var].attrs[key] = json.dumps(value)
                elif isinstance(value, bool):
                    # Convert boolean to integer
                    ds[var].attrs[key] = int(value)
    
    return ds

Group by month and write each to a separate file